# Quantum Espresso Energy Volume Curve Workflow with aiida

## Define workflow with aiida

In [ ]:
from python_workflow_definition.aiida import write_workflow_json
from python_workflow_definition.shared import get_dict, get_list

from aiida import load_profile

load_profile()

workflow_json_filename = "aiida_qe.json"

In [ ]:
from aiida_workgraph import task, WorkGraph

In [ ]:
@task.pythonjob()
def pickle_node(value):
    """Handle data nodes"""
    return value

In [ ]:
from quantum_espresso_workflow import generate_structures as _generate_structures
from quantum_espresso_workflow import get_bulk_structure as _get_bulk_structure
from quantum_espresso_workflow import calculate_qe as _calculate_qe
from quantum_espresso_workflow import (
    plot_energy_volume_curve as _plot_energy_volume_curve,
)

In [ ]:
strain_lst = [0.9, 0.95, 1.0, 1.05, 1.1]

In [ ]:
get_bulk_structure = task.pythonjob()(_get_bulk_structure)
generate_structures = task.pythonjob()(_generate_structures)
calculate_qe = task.pythonjob(outputs=["energy", "volume", "structure"])(_calculate_qe)
plot_energy_volume_curve = task.pythonjob()(_plot_energy_volume_curve)

In [ ]:
wg = WorkGraph("wg-qe")

## Helper tasks that just pickle input data

In [ ]:
pickle_element_task = wg.add_task(
    pickle_node,
    name="pickle_element",
    value="Al",
)

In [ ]:
pickle_a_task = wg.add_task(pickle_node, name="pickle_a", value=4.05)

In [ ]:
pickle_cubic_task = wg.add_task(pickle_node, name="pickle_cubic", value=True)

In [ ]:
pickle_relax_workdir_task = wg.add_task(
    pickle_node,
    name="pickle_relax_workdir",
    value="mini",
)

In [ ]:
pickle_pp_task = wg.add_task(
    pickle_node,
    name="pseudopotentials",
    value={"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"},
)

In [ ]:
pickle_kpts_task = wg.add_task(pickle_node, name="kpts_task", value=[3, 3, 3])

In [ ]:
pickle_calc_type_relax_task = wg.add_task(
    pickle_node,
    name="calc_type_relax",
    value="vc-relax",
)

In [ ]:
pickle_calc_type_scf_task = wg.add_task(
    pickle_node,
    name="calc_type_scf",
    value="scf",
)

In [ ]:
pickle_smearing_task = wg.add_task(pickle_node, name="smearing", value=0.02)

In [ ]:
pickle_strain_lst_task = wg.add_task(
    pickle_node,
    name="pickle_strain_lst",
    value=strain_lst,
)

## Actual tasks to construct the EOS workflow

In [ ]:
get_bulk_structure_task = wg.add_task(
    get_bulk_structure,
    name="get_bulk_structure",
    register_pickle_by_value=True,
    element=pickle_element_task.outputs.result,
    a=pickle_a_task.outputs.result,
    cubic=pickle_cubic_task.outputs.result,
)

In [ ]:
relax_prepare_input_dict_task = wg.add_task(
    task.pythonjob()(get_dict),
    name="relax_get_dict",
    register_pickle_by_value=True,
    structure=get_bulk_structure_task.outputs.result,
    calculation=pickle_calc_type_relax_task.outputs.result,
    kpts=pickle_kpts_task.outputs.result,
    pseudopotentials=pickle_pp_task.outputs.result,
    smearing=pickle_smearing_task.outputs.result,
)

relax_task = wg.add_task(
    calculate_qe,
    name="mini",
    register_pickle_by_value=True,
    input_dict=relax_prepare_input_dict_task.outputs.result,
    working_directory=pickle_relax_workdir_task.outputs.result,
)

In [ ]:
generate_structures_task = wg.add_task(
    generate_structures,
    name="generate_structures",
    register_pickle_by_value=True,
    structure=relax_task.outputs.structure,
    strain_lst=pickle_strain_lst_task.outputs.result,
)

In [ ]:
get_volumes_task = wg.add_task(
    task.pythonjob()(get_list),
    name="get_volumes",
    register_pickle_by_value=True,
)

In [ ]:
get_energies_task = wg.add_task(
    task.pythonjob()(get_list),
    name="get_energies",
    register_pickle_by_value=True,
)

In [ ]:
strain_dir_tasks, scf_qe_tasks, scf_get_dict_tasks = [], [], []

for i, strain in enumerate(strain_lst):

    structure_key = f"s_{i}"
    strain_dir = f"strain_{i}"
    generate_structures_task.add_output("workgraph.any", structure_key)

    strain_dir_task = wg.add_task(
        pickle_node,
        name=f"pickle_{strain_dir}_dir",
        value=strain_dir,
        register_pickle_by_value=True,
    )

    scf_prepare_input_dict_task = wg.add_task(
        task.pythonjob()(get_dict),
        name=f"get_dict_{i}",
        register_pickle_by_value=True,
        structure=generate_structures_task.outputs[structure_key],
        calculation=pickle_calc_type_scf_task.outputs.result,
        kpts=pickle_kpts_task.outputs.result,
        pseudopotentials=pickle_pp_task.outputs.result,
        smearing=pickle_smearing_task.outputs.result,
    )

    scf_qe_task = wg.add_task(
        calculate_qe,
        name=f"qe_{i}",
        register_pickle_by_value=True,
        input_dict=scf_prepare_input_dict_task.outputs.result,
        working_directory=strain_dir_task.outputs.result,
    )

    # collect energy and volume
    get_energies_task.set({f"{i}": scf_qe_task.outputs.energy})
    get_volumes_task.set({f"{i}": scf_qe_task.outputs.volume})

In [ ]:
plot_energy_volume_curve_task = wg.add_task(
    plot_energy_volume_curve,
    name="plot_energy_volume_curve",
    register_pickle_by_value=True,
    volume_lst=get_volumes_task.outputs.result,
    energy_lst=get_energies_task.outputs.result,
)

In [ ]:
wg

In [ ]:
_ = write_workflow_json(wg=wg, file_name=workflow_json_filename)


In [ ]:
!cat {workflow_json_filename}

## Load Workflow with jobflow

In [ ]:
from python_workflow_definition.jobflow import load_workflow_json

In [ ]:
from jobflow.managers.local import run_locally

In [ ]:
flow = load_workflow_json(file_name=workflow_json_filename)

In [ ]:
result = run_locally(flow)
result

## Load Workflow with pyiron_base

In [ ]:
from pyiron_base import Project

In [ ]:
from python_workflow_definition.pyiron_base import load_workflow_json

In [ ]:
pr = Project("test")
pr.remove_jobs(recursive=True, silently=True)

In [ ]:
delayed_object = load_workflow_json(project=pr, file_name=workflow_json_filename)
delayed_object.draw()

In [ ]:
delayed_object.pull()